In [20]:
# import requests
# import os
# import cv2
# import numpy as np

# os.environ["no_proxy"] = "localhost,172.16.87.75"

# # API endpoint
# url = "http://localhost:8000/lip_sync"

# # Prepare the data
# try:
#     video_file = open('data/video/sun.mp4', 'rb')
#     audio_file = open('data/audio/sun.wav', 'rb')
# except FileNotFoundError as e:
#     print(f"Error: {e}")
#     exit(1)

# files = {
#     'video': ('sun.mp4', video_file, 'video/mp4'),
#     'audio': ('sun.wav', audio_file, 'audio/wav'),
# }

# data = {
#     'avatar_id': 'avator_1',
#     'fps': 25,
#     'batch_size': 4
# }

# try:
#     # Send the POST request
#     response = requests.post(url, files=files, data=data, stream=True)

#     # Output video path
#     output_video_path = 'output_video.mp4'

#     # Check if the request was successful
#     if response.status_code == 200:
#         # Create a video writer (adjust codec and parameters as needed)
#         fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'XVID' for .avi
#         out = None 

#         # Create a bytes buffer for the image data
#         bytes_buffer = bytes()

#         for chunk in response.iter_content(chunk_size=1024):
#             if chunk:
#                 bytes_buffer += chunk
#                 a = bytes_buffer.find(b'\xff\xd8')  # JPEG start
#                 b = bytes_buffer.find(b'\xff\xd9')  # JPEG end
#                 if a != -1 and b != -1:
#                     jpg = bytes_buffer[a:b+2]
#                     bytes_buffer = bytes_buffer[b+2:]
#                     frame = cv2.imdecode(np.frombuffer(jpg, dtype=np.uint8), cv2.IMREAD_COLOR)
                    
#                     # Initialize the video writer with frame size
#                     if out is None and frame is not None:
#                         height, width, _ = frame.shape
#                         out = cv2.VideoWriter(output_video_path, fourcc, data['fps'], (width, height))

#                     # Write the frame to the video writer
#                     if out is not None:
#                         out.write(frame)

#         # Release the video writer
#         if out is not None:
#             out.release()

#         print(f"Lip-synced video saved as {output_video_path}")
#     else:
#         print(f"Error: {response.status_code}")
#         print(response.text)
# finally:
#     # Close the files
#     video_file.close()
#     audio_file.close()

Lip-synced video saved as output_video.mp4


In [3]:
import os
import requests

# Define the FastAPI server URL
base_url = "http://localhost:8000"

os.environ["no_proxy"] = "localhost,172.16.87.75"

# 1. Check Server Status
response = requests.get(f"{base_url}/digital_human/check")
if response.status_code == 200:
    print("Server is enabled:", response.json())

# 2. Make a POST request to /digital_human/gen
digital_human_data = {
    "user_id": "user123",
    "request_id": "req123",
    "streamer_id": "1",
    "tts_path": "./data/audio/elon.wav",
    "chunk_id": 0
}

response = requests.post(f"{base_url}/digital_human/gen", json=digital_human_data)
if response.status_code == 200:
    print("Generated digital human video:", response.json())
else:
    print("Failed to generate digital human:", response.text)

# # 3. Make a POST request to /digital_human/preprocess
# preprocess_data = {
#     "user_id": "user123",
#     "request_id": "req123",
#     "streamer_id": "streamer1",
#     "video_path": "/path/to/video.mp4"
# }

# response = requests.post(f"{base_url}/digital_human/preprocess", json=preprocess_data)
# if response.status_code == 200:
#     print("Preprocessing complete:", response.json())
# else:
#     print("Failed to preprocess video:", response.text)


Server is enabled: {'message': 'server enabled'}
Generated digital human video: {'user_id': 'user123', 'request_id': 'req234', 'digital_human_mp4_path': '/space/hotel/phit/personal/rtlipsync/results/avatars/vid_output/req234.mp4'}


Take 11m29s to solve this problem.

In [4]:
import os
import requests

# Define the FastAPI server URL
base_url = "http://localhost:8000"

os.environ["no_proxy"] = "localhost,172.16.87.75"

# 1. Check Server Status
response = requests.get(f"{base_url}/digital_human/check")
if response.status_code == 200:
    print("Server is enabled:", response.json())

# 2. Make a POST request to /digital_human/gen
digital_human_data = {
    "user_id": "user123",
    "request_id": "req345",
    "streamer_id": "1",
    "tts_path": "./data/audio/elon.wav",
    "chunk_id": 0
}

response = requests.post(f"{base_url}/digital_human/gen", json=digital_human_data)
if response.status_code == 200:
    print("Generated digital human video:", response.json())
else:
    print("Failed to generate digital human:", response.text)

# # 3. Make a POST request to /digital_human/preprocess
# preprocess_data = {
#     "user_id": "user123",
#     "request_id": "req123",
#     "streamer_id": "streamer1",
#     "video_path": "/path/to/video.mp4"
# }

# response = requests.post(f"{base_url}/digital_human/preprocess", json=preprocess_data)
# if response.status_code == 200:
#     print("Preprocessing complete:", response.json())
# else:
#     print("Failed to preprocess video:", response.text)


Server is enabled: {'message': 'server enabled'}
Generated digital human video: {'user_id': 'user123', 'request_id': 'req345', 'digital_human_mp4_path': '/space/hotel/phit/personal/rtlipsync/results/avatars/vid_output/req345.mp4'}


In [23]:
# !curl -x "" -X POST http://172.16.87.75:8010/humanaudio -F "file=@data/video/elon.mp4" -F "sessionid=0"

import os
import httpx

os.environ["no_proxy"] = "localhost,172.16.87.75,127.0.0.1"

url = "http://172.16.87.75:8010/humanaudio"
files = {
    'file': open('/space/hotel/phit/personal/rtlipsync/data/audio/elon.wav', 'rb'),
    'sessionid': (None, '0')
}

response = httpx.post(url, files=files)

print(response.text)

{"code":-1,"msg":"err","data":"list index out of range"}


In [1]:
import asyncio
import websockets
import json
import aiohttp
import pyaudio
import wave
import av
from aiortc import RTCPeerConnection, RTCSessionDescription, VideoStreamTrack
from aiortc.contrib.media import MediaPlayer
from aiortc import MediaStreamTrack

import os

os.environ["no_proxy"] = "localhost,172.16.87.75,127.0.0.1"


class AudioStreamTrack(MediaStreamTrack):
    kind = "audio"

    def __init__(self, track):
        super().__init__()
        self.track = track

    async def recv(self):
        frame = await self.track.recv()
        return frame

async def send_offer(session, url):
    pc = RTCPeerConnection()
    
    # Add audio track
    audio_track = AudioStreamTrack(MediaPlayer('data/audio/elon.wav').audio)
    pc.addTrack(audio_track)

    # Create and set local description
    await pc.setLocalDescription(await pc.createOffer())

    # Send the offer to the server
    async with session.post(f'{url}/offer', json={
        'sdp': pc.localDescription.sdp,
        'type': pc.localDescription.type
    }) as response:
        answer = await response.json()
        await pc.setRemoteDescription(RTCSessionDescription(sdp=answer['sdp'], type=answer['type']))
        return answer['sessionid'], pc

async def send_audio_data(websocket, sessionid, audio_data):
    await websocket.send(json.dumps({
        'type': 'audio',
        'sessionid': sessionid,
        'data': audio_data.tobytes().decode('latin-1')  # Convert bytes to string
    }))

async def receive_video(pc, output_file):
    # Set up a video recorder
    recorder = av.open(output_file, 'w')
    video_stream = recorder.add_stream('h264', rate=30)
    
    @pc.on("track")
    async def on_track(track):
        if track.kind == "video":
            while True:
                frame = await track.recv()
                # Encode and write the frame
                packet = video_stream.encode(frame)
                if packet:
                    recorder.mux(packet)

async def main():
    url = "http://localhost:8010"  # Replace with your server's address
    ws_url = "ws://localhost:8010/ws"  # WebSocket URL

    async with aiohttp.ClientSession() as session:
        # Establish WebRTC connection
        sessionid, pc = await send_offer(session, url)

        # Set up WebSocket connection
        async with websockets.connect(ws_url) as websocket:
            # Set up PyAudio for microphone input
            p = pyaudio.PyAudio()
            stream = p.open(format=pyaudio.paInt16, channels=1, rate=44100, input=True, frames_per_buffer=1024)

            # Start receiving video
            video_task = asyncio.create_task(receive_video(pc, 'output_video.mp4'))

            try:
                while True:
                    # Read audio data from microphone
                    audio_data = stream.read(1024)
                    # Send audio data through WebSocket
                    await send_audio_data(websocket, sessionid, audio_data)
            except KeyboardInterrupt:
                print("Stopping...")
            finally:
                # Clean up
                stream.stop_stream()
                stream.close()
                p.terminate()
                await pc.close()
                await video_task

if __name__ == "__main__":
    asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop